## Problem Statement

Our customer, Don Francesco, manages a large jewelry store and seeks to use a comprehensive dataset he has gathered to automatically determine the prices for diamonds customers want to sell to him.

This dataset includes details on the characteristics and prices of diamonds, which closely reflect actual market values.

Our goal is to develop a model that can accurately predict the market price of a diamond based on its characteristics.

### Dataset
The dataset provides information on diamond prices and their attributes that impact their value, expressed in 2008 US Dollars.

Key attributes include the 4 Cs: `carat`, `cut`, `color`, and `clarity`. It also includes physical measurements such as `depth`, `table`, and dimensions (`x`, `y`, `z`).

Additional information is available in the dataset readme.

### Caution 💀💀💀
I'm confident that my analysis is accurate and the model is ready for deployment. However, I had to write the code quickly and some sections might not be up to standard. I apologize for this. Please ensure the codebase is thoroughly refined and optimized.

## Data Understanding

## Data Exploration & Preparation


### Importing data
We base all our analysis on a CSV export of Don Francesco's database. There may be other samples, but that's all we were able to get.

In [ ]:
import pandas as pd

diamonds = pd.read_csv("https://raw.githubusercontent.com/xtreamsrl/xtream-ai-assignment-engineer/main/datasets/diamonds/diamonds.csv")
diamonds.head()

First, let us check if there are really no missing data.

In [ ]:
diamonds.isna().sum()

Great! No missing data.

Let us check the summary of the dataset.

In [ ]:
diamonds.describe()

There is something wrong. Negative prices and zero-dimensional stones are the result of mistakes.

In [ ]:
diamonds[diamonds.x * diamonds.y * diamonds.z == 0]

In [ ]:
diamonds[diamonds.price <= 0]

That's not good. Let us remove those samples.

In [ ]:
diamonds = diamonds[(diamonds.x * diamonds.y * diamonds.z != 0) & (diamonds.price > 0)]

Let us check again.

In [ ]:
diamonds.describe()

Good, no more strange values. It looks like the distributions of the numerical variables are quite skewed: there are a few very big values and lots of smaller ones. Let us take a look to the charts.


In [ ]:
import numpy as np
from pandas.plotting import scatter_matrix

scatter_matrix(diamonds.select_dtypes(include=['number']), figsize=(14, 10));

We see several interesting things:
1. There are variables which does not look very correlated with the target (e.g table)
2. There are variables which look very correlated with the target, like carat, x, y and z, with non-linear patterns
3. There are variables which look correlated with each other (e.g. x, y and z)




Then, let us look into the distribution of single variables. We can use histograms.

In [ ]:
diamonds.hist(bins=100, figsize=(14, 10));

There are some outliers and it may be a good idea to remove them, but we will not do that at the moment.

Let us explore categorical variables: cut, color and clarity.

We can use box or violin charts.

In [ ]:
import plotly.express as px

def plot_diamonds_price_by(diamonds_df, cut_column):
  return px.violin(diamonds_df, x=cut_column, y='price', color=cut_column, title=f'Price by {cut_column}')

In [ ]:
plot_diamonds_price_by(diamonds, 'cut')

In [ ]:
plot_diamonds_price_by(diamonds, 'color')

In [ ]:
plot_diamonds_price_by(diamonds, 'clarity')

The distribution of the price differs with the values of each categorical variable. Therefore, it looks like all the variables may add relevant information.

To assess relationships between multiple variables, we need to map each one to a different graphical element, or aesthetic.

For instance, we can perform a scatter of price vs weight, with the categorical varaibles cut, color and clarity as color.

In [ ]:
def scatter_diamods_by(diamonds_df, cut_column):
  return px.scatter(diamonds_df, x='carat', y='price', color=cut_column, title=f'Price vs carat with {cut_column}')

In [ ]:
scatter_diamods_by(diamonds, 'cut')

In [ ]:
scatter_diamods_by(diamonds, 'clarity')

In [ ]:
scatter_diamods_by(diamonds, 'color')

The scatter plots confirm that all the categorical variables are relevant because, when we control for the weight of the stone, the categorical variables explain part of the remaining variance in the price.


## Linear Model

We'll begin with our preferred baseline: a straightforward, fully explainable linear model. However, we need to apply some transformations to the dataset before training the model.

### Data Preparation

First, we are going to drop all the irrelevant columns, namely depth and table.

Moreover, we also want to drop y and z, as they have high correlation with x.

In [ ]:
diamonds_processed = diamonds.drop(columns=['depth', 'table', 'y', 'z'])
diamonds_processed.head()

Then, we are going to create dummy variables for cut, color and clarity.

Pandas has a very handy function for that.

In [ ]:
diamonds_dummy = pd.get_dummies(diamonds_processed, columns=['cut', 'color', 'clarity'], drop_first=True)
diamonds_dummy.head()

Finally, we split X and Y, train and test.

We go for a random 80/20 split.

In [ ]:
from sklearn.model_selection import train_test_split

x = diamonds_dummy.drop(columns='price')
y = diamonds_dummy.price

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

### Modelling & evaluation

We will use a simple linear regression model.

We train the model and we evaluate its out-of-sample performances using r squared and Mean Absolute Error (MAE).

In [ ]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg.fit(x_train, y_train)
pred = reg.predict(x_test)

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error

print(f'R2 Score: {round(r2_score(y_test, pred), 4)}')
print(f'MAE: {round(mean_absolute_error(y_test, pred), 2)}$')

Let us visualize our results in a goodness of fit plot.

In [ ]:
import matplotlib.pyplot as plt

def plot_gof(y_true: pd.Series, y_pred: pd.Series):
  plt.plot(y_true, y_pred, '.')
  plt.plot(y_true, y_true, linewidth=3, c='black')
  plt.xlabel('Actual')
  plt.ylabel('Predicted')
  plt.show()

plot_gof(y_test, pred)

That's not good at all. We have some negative predicted prices.

To avoid this issue, we can perform a log transformation on the target variable.

In [ ]:
y_train_log = np.log(y_train)

reg = LinearRegression()
reg.fit(x_train, y_train_log)
pred_log = reg.predict(x_test)
pred = np.exp(pred_log)

And we can check the same metrics as before.


In [ ]:
print(f'R2 Score: {round(r2_score(y_test, pred), 4)}')
print(f'MAE: {round(mean_absolute_error(y_test, pred), 2)}$')

Much, much better. Again, we can visually assess the improvement.

In [ ]:
plot_gof(y_test, pred)

## Gradient boosting
The linear regression is fun and simple, but more advanced models may be needed in order to achieve better performance. Let us try with xgboost.

### Data Preparation
We know that tree-based models do not suffer from collinear variables and prefer ordinal variables to categorical ones. Therefore, we need to change the preprocessing as well.

In [ ]:
diamonds_processed_xgb = diamonds.copy()
diamonds_processed_xgb['cut'] = pd.Categorical(diamonds_processed_xgb['cut'], categories=['Fair', 'Good', 'Very Good', 'Ideal', 'Premium'], ordered=True)
diamonds_processed_xgb['color'] = pd.Categorical(diamonds_processed_xgb['color'], categories=['D', 'E', 'F', 'G', 'H', 'I', 'J'], ordered=True)
diamonds_processed_xgb['clarity'] = pd.Categorical(diamonds_processed_xgb['clarity'], categories=['IF', 'VVS1', 'VVS2', 'VS1', 'VS2', 'SI1', 'SI2', 'I1'], ordered=True)
diamonds_processed_xgb.info()

By using the same random seed, we ensure that the same samples end up in the test set, therefore the comparison between models is fair.

In [ ]:
x_train_xbg, x_test_xbg, y_train_xbg, y_test_xbg = train_test_split(diamonds_processed_xgb.drop(columns='price'), diamonds_processed_xgb['price'], test_size=0.2, random_state=42)

### Modelling and Evaluation
As before, we train the model and we evaluate it with the same metrics.

In [ ]:
import xgboost

xgb = xgboost.XGBRegressor(enable_categorical=True, random_state=42)
xgb.fit(x_train_xbg, y_train_xbg)
xgb_pred = xgb.predict(x_test_xbg)

In [ ]:
print(f'R2 Score: {round(r2_score(y_test_xbg, xgb_pred), 4)}')
print(f'MAE: {round(mean_absolute_error(y_test_xbg, xgb_pred), 2)}$')

In [ ]:
plot_gof(y_test_xbg, xgb_pred)

Much, much better. But we can do even better. Let's try and use optuna, a Bayesian hyperparameter tuning library.

In [ ]:
!pip install --upgrade optuna

In [ ]:
import optuna

def objective(trial: optuna.trial.Trial) -> float:
    # Define hyperparameters to tune
    param = {
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3, 0.4, 0.5, 0.7]),
        'subsample': trial.suggest_categorical('subsample', [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
        'learning_rate': trial.suggest_float('learning_rate', 1e-8, 1.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'random_state': 42,
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'enable_categorical': True
    }

    # Split the training data into training and validation sets
    x_train, x_val, y_train, y_val = train_test_split(x_train_xbg, y_train_xbg, test_size=0.2, random_state=42)

    # Train the model
    model = xgboost.XGBRegressor(**param)
    model.fit(x_train, y_train)

    # Make predictions
    preds = model.predict(x_val)

    # Calculate MAE
    mae = mean_absolute_error(y_val, preds)

    return mae

study = optuna.create_study(direction='minimize', study_name='Diamonds XGBoost')
study.optimize(objective, n_trials=100)
print("Best hyperparameters: ", study.best_params)

Let's now re-train the model with the best set of hyperparameters.

In [ ]:
xgb_opt = xgboost.XGBRegressor(**study.best_params, enable_categorical=True, random_state=42)
xgb_opt.fit(x_train_xbg, y_train_xbg)
xgb_opt_pred = xgb_opt.predict(x_test_xbg)

In [ ]:
print(f'R2 Score: {round(r2_score(y_test_xbg, xgb_opt_pred), 4)}')
print(f'MAE: {round(mean_absolute_error(y_test_xbg, xgb_opt_pred), 2)}$')

In [ ]:
plot_gof(y_test_xbg, xgb_pred)

A modest improvement. The model performs well, especially for smaller and less expensive gems. However, it shows larger errors with bigger stones, which warrants further investigation. For now, this is acceptable.